<a href="https://colab.research.google.com/github/rish4/NewColonoscopy/blob/main/New_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
from keras.preprocessing import image
from IPython.display import display
from PIL import Image
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

In [6]:
## Set file paths to image files

train_path = "/content/drive/MyDrive/NewDB/Training set"
test_path = "/content/drive/MyDrive/NewDB/Test set"

## Set up hyperparameters that will be used later
hyper_dimension = 64
hyper_batch_size = 32
hyper_epochs = 100
hyper_channels = 3
hyper_mode = 'rgb'



In [7]:

'''cnn_model = cnn.fit_generator(train_generator, 
                              steps_per_epoch = len(train_generator), 
                              epochs = 20, 
                              validation_data = test_generator,
                              validation_steps = len(test_generator)
                              )'''

'cnn_model = cnn.fit_generator(train_generator, \n                              steps_per_epoch = len(train_generator), \n                              epochs = 20, \n                              validation_data = test_generator,\n                              validation_steps = len(test_generator)\n                              )'

In [15]:
def create_model():
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.727273 using {'batch_size': 40, 'epochs': 100}
0.536364 (0.137470) with: {'batch_size': 10, 'epochs': 10}
0.563636 (0.078554) with: {'batch_size': 10, 'epochs': 50}
0.593939 (0.111423) with: {'batch_size': 10, 'epochs': 100}
0.506061 (0.142069) with: {'batch_size': 20, 'epochs': 10}
0.657576 (0.080060) with: {'batch_size': 20, 'epochs': 50}
0.560606 (0.119303) with: {'batch_size': 20, 'epochs': 100}
0.596970 (0.104006) with: {'batch_size': 40, 'epochs': 10}
0.660606 (0.148701) with: {'batch_size': 40, 'epochs': 50}
0.727273 (0.222681) with: {'batch_size': 40, 'epochs': 100}
0.566667 (0.101323) with: {'batch_size': 60, 'epochs': 10}
0.727273 (0.222681) with: {'batch_size': 60, 'epochs': 50}
0.660606 (0.148701) with: {'batch_size': 60, 'epochs': 100}
0.593939 (0.111423) with: {'batch_size': 80, 'epochs': 10}
0.663636 (0.182876) with: {'batch_size': 80, 'epochs': 50}
0.627273 (0.122643) with: {'batc